In [1]:
import numpy as np
import pandas as pd

In [2]:
dataframe = pd.read_csv('data/train.csv')
dataframe.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,18,1
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,1,0


In [3]:
X = dataframe.iloc[:, [3, 5, 6, 7, 8, 9, 10, 11, 12]].values
y = dataframe.iloc[:, -1]
index = 0
print('X: ',X[index, :])
print('y: ',y[index])

X:  [20.0 False 'US' 'USD' 1241333999 1241334017 1240600507 1240602723 3]
y:  1


In [4]:
# data processing

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder_disable_com = LabelEncoder()
X[:, 1] = label_encoder_disable_com.fit_transform(X[:, 1])

label_encoder_country = LabelEncoder()
X[:, 2] = label_encoder_country.fit_transform(X[:, 2])

label_encoder_currency = LabelEncoder()
X[:, 3] = label_encoder_currency.fit_transform(X[:, 3])

index = 1
print('Before hot encoding: ', X[index,:])
print('Shape before hot encoding: ', X.shape)

# one hot encoding in last for multiple columns
one_hot_encoding_country = OneHotEncoder(categorical_features = [1, 2, 3])
X = one_hot_encoding_country.fit_transform(X).toarray()
print('After hot encoding: ', X[index,:])
print('Shape after hot encoding: ', X.shape)

Before hot encoding:  [300.0 0 10 8 1242429000 1242432018 1240960224 1240975592 2]
Shape before hot encoding:  (108129, 9)
After hot encoding:  [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 3.00000000e+02 1.24242900e+09
 1.24243202e+09 1.24096022e+09 1.24097559e+09 2.00000000e+00]
Shape after hot encoding:  (108129, 28)


In [6]:
# Train test split for training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape)

(86503, 28) (21626, 28)


In [7]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
print('After feature scaling: ', X_train[10])

After feature scaling:  [ 0.05490662 -0.05490662 -0.13328979 -0.18892113 -0.00340006 -0.04304735
 -0.29679631 -0.03384938 -0.08115553 -0.03333198 -0.05718973 -0.04666978
  0.41837912 -0.13328979 -0.18892113 -0.04304735 -0.08808457 -0.29679631
 -0.03333198 -0.05718973 -0.04666978  0.41837912 -0.00735224  0.47601584
  0.47866315  0.47539681  0.47160975 -0.09232643]


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

classifier = Sequential()

input_dim = int(X_train.shape[1])
output_dim = int(X_train.shape[1] / 2)
print('Input dim: ', input_dim)
print('Output dim: ', output_dim)

#input layer
classifier.add(Dense(output_dim=output_dim, init='uniform', activation='relu', input_dim=input_dim))
classifier.add(Dropout(p=0.1))

# hidden layer
classifier.add(Dense(output_dim=output_dim, init='uniform', activation='relu'))
classifier.add(Dropout(p=0.1))

# output layer
classifier.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

# compile ann
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# fit ANN
classifier.fit(X_train, y_train, batch_size=50, nb_epoch=50)


/home/zahirah/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Input dim:  28
Output dim:  14


/home/zahirah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=28, units=14, kernel_initializer="uniform")`
  
/home/zahirah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  from ipykernel import kernelapp as app
/home/zahirah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=14, kernel_initializer="uniform")`
/home/zahirah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
/home/zahirah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
/home/zahirah/a

Epoch 1/50
86503/86503 [==============================] - 5s 57us/step - loss: 0.5599 - acc: 0.7194
Epoch 2/50
86503/86503 [==============================] - 3s 36us/step - loss: 0.4246 - acc: 0.8024
Epoch 3/50
86503/86503 [==============================] - 3s 31us/step - loss: 0.3877 - acc: 0.8149
Epoch 4/50
86503/86503 [==============================] - 6s 69us/step - loss: 0.3730 - acc: 0.8203
Epoch 5/50
86503/86503 [==============================] - 6s 72us/step - loss: 0.3669 - acc: 0.8257
Epoch 6/50
86503/86503 [==============================] - 6s 71us/step - loss: 0.3580 - acc: 0.8318
Epoch 7/50
86503/86503 [==============================] - 3s 37us/step - loss: 0.3541 - acc: 0.8355
Epoch 8/50
86503/86503 [==============================] - 3s 31us/step - loss: 0.3505 - acc: 0.8375
Epoch 9/50
86503/86503 [==============================] - 5s 62us/step - loss: 0.3489 - acc: 0.8399
Epoch 10/50
86503/86503 [==============================] - 7s 76us/step - loss: 0.3472 - acc: 0.8412

KeyboardInterrupt: 

In [10]:
# Prediction with trained classifier
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[ True],
       [ True],
       [ True],
       ...,
       [False],
       [ True],
       [False]])

In [12]:
# Classifier Metrics

from sklearn.metrics import accuracy_score
metrics = accuracy_score(y_test, y_pred)
metrics


0.8603070378248404